In [2]:
import numpy as np
import pickle
from funcy                     import *
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers        import l2
from keras.utils               import plot_model
import keras.layers as kl

import psutil
import keras


# basic configs
weight_decay = 0.0005 
use_bias = False
weight_init="he_normal" 
padding = 'same'
L2 = keras.regularizers.l2(weight_decay)
conv_config = {'padding':padding,
               'kernel_initializer':weight_init,
               'kernel_regularizer':L2,
               'use_bias':use_bias}

# insuring image data channel is at last axis 
keras.backend.image_data_format()
# keras.backend..set_image_data_format('channels_first')

Using CNTK backend
c:\users\kojima\appdata\local\programs\python\python35\lib\site-packages\keras\backend\cntk_backend.py:21: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


'channels_last'

In [3]:
# Utility functions.

def ljuxt(*fs):  
    return rcompose(juxt(*fs), list)

def bn():return kl.BatchNormalization()

def relu():return kl.Activation('relu')

def conv(filter_size, kernel_size, stride_size=1):
    return kl.Conv2D(filters=filter_size, kernel_size=kernel_size,strides=stride_size, **conv_config)

def conv3(filter_size):return conv(filter_size, kernel_size=(3,3), stride_size=1)
def conv1(filter_size):return conv(filter_size, kernel_size=(1,1), stride_size=1)

def add():return kl.Add()

def GlobalAvgPool():return kl.GlobalAveragePooling2D()

def avgPool(pool_size=2,strides=2):return kl.AveragePooling2D(pool_size=pool_size,strides=strides)

def dense(unit_size, activation):
    return kl.Dense(unit_size, activation=activation, kernel_regularizer=L2)  

# original layer which is pad the channel size with zeros matrix, 
# ex. padding (None,28,28,1) 2 zeros channels as (None,28,28,3)
class zeroPad(keras.engine.topology.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        self.l = None
        super(zeroPad, self).__init__(**kwargs)

    def build(self, input_shape):
        super(zeroPad, self).build(input_shape)

    def call(self, x):
        x_shape = x.shape.as_list()
        if self.output_dim == x_shape[-1]:
            return x
        if self.l is None:
            padding = self.output_dim//x_shape[-1]
            self.l = [x] + [kl.K.zeros_like(x) for i in range(padding-1)]
            self.l +=  [kl.K.zeros_like(x)[:,:,:,:self.output_dim - padding*x_shape[-1]] ]
        return kl.concatenate(self.l)

    def compute_output_shape(self, input_shape):
        return (input_shape[0],input_shape[1],input_shape[2], self.output_dim)

    
from keras.engine import Layer, InputSpec
try:
    from keras import initializations
except ImportError:
    from keras import initializers as initializations
import keras.backend as K
class Scale(Layer):
    '''Custom Layer for DenseNet used for BatchNormalization.
    
    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases larned.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializations.get(beta_init)
        self.gamma_init = initializations.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        # Tensorflow >= 1.0.0 compatibility
        self.gamma = K.variable(self.gamma_init(shape), name='{}_gamma'.format(self.name))
        self.beta = K.variable(self.beta_init(shape), name='{}_beta'.format(self.name))
        #self.gamma = self.gamma_init(shape, name='{}_gamma'.format(self.name))
        #self.beta = self.beta_init(shape, name='{}_beta'.format(self.name))
        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# dense net architecture
![title](https://cdn-images-1.medium.com/max/1600/1*KOjUX1ST5RnDOZWWLWRGkw.png)
![title](https://data-sci.info/wp-content/uploads/2017/07/densenet-4.png)

In [4]:
def bn_scale_relu_conv(compression=1.0, kernel_size=1, stride_size=1):
    def f(x,compression=compression):
        x= rcompose(bn(),
#                     Scale(),
                    relu(),
                    conv(int(int(x.shape[-1]) * compression), kernel_size, stride_size),
                   )(x)
        return x
    return f

def dense_unit(growth,units = 4):
    def f(x):
        ls = x
        for i in range(units):
            x = rcompose(bn(),#Scale(),
                         relu(),conv3(growth*4),
                         bn(),#Scale(),
                         relu(),conv3(growth),)(x)
            ls = x = kl.Concatenate()([x,ls])
        return x
    return f

def denseNet(class_size=1000,first=2**5,growth=16,compression=1/4,units=[6,12,24,16]):
    units = iter(units)
    return rcompose(conv3(first),
                    
                    dense_unit(growth,next(units)),
                    bn_scale_relu_conv(compression),
                    kl.AveragePooling2D(2,2),
                    
                    dense_unit(growth,next(units)),
                    bn_scale_relu_conv(compression),
                    kl.AveragePooling2D(2,2),
                    
                    dense_unit(growth,next(units)),
                    bn_scale_relu_conv(compression),
                    kl.AveragePooling2D(2,2),
                    
                    dense_unit(growth,next(units)),
                    bn_scale_relu_conv(compression),
                    kl.AveragePooling2D(2,2),
                    
                    bn(),Scale(),relu(),
                    GlobalAvgPool(),
                    dense(class_size, 'softmax')
    )

def bn_scale_relu_updim_conv(compression=1.0, kernel_size=1, stride_size=1):
    def f(x,compression=compression):
        x= rcompose(bn(),
#                     Scale(),
                    relu(),
                    kl.UpSampling2D(2),
                    conv(int(int(x.shape[-1]) * compression), kernel_size, stride_size),
                   )(x)
        return x
    return f

def denseNet_FCN(class_size=1000,first=2**5,growth=2,compression=1/4,units=[6,12,24,16]):
    units_fcn = iter(units[::-1])
    units = iter(units)
    
    return rcompose(conv3(first),
                    
                    dense_unit(growth,next(units)),
                    bn_scale_relu_conv(compression),
                    kl.AveragePooling2D(2,2),
                    
                    dense_unit(growth,next(units)),
                    bn_scale_relu_conv(compression),
                    kl.AveragePooling2D(2,2),
                    
                    dense_unit(growth,next(units)),
                    bn_scale_relu_conv(compression),
                    kl.AveragePooling2D(2,2),
                    
                    dense_unit(growth,next(units)),
                    bn_scale_relu_conv(compression),
                    kl.AveragePooling2D(2,2),
                    
                    #=====================================
                    
                    bn_scale_relu_updim_conv(compression),
                    dense_unit(growth,next(units_fcn)),
                    
                    bn_scale_relu_updim_conv(compression),
                    dense_unit(growth,next(units_fcn)),
                    
                    bn_scale_relu_updim_conv(compression),
                    dense_unit(growth,next(units_fcn)),
                    
                    bn_scale_relu_updim_conv(compression),
                    dense_unit(growth,next(units_fcn)),
                    
                    conv1(class_size),
                    kl.Activation('sigmoid')
#                     bn(),Scale(),relu(),
#                     GlobalAvgPool(),
#                     dense(class_size, 'softmax')
    )

In [ ]:
path = 'I:/Downloads/unet-master/npydata/'#'F:/Installer/unet-master/npydata/'
x_train=np.load(path+'imgs_train.npy')/255.0
x_test=np.load(path+'imgs_test.npy')/255.0
y_train=np.round(np.load(path+'imgs_mask_train.npy')/255.0)

In [ ]:
from skimage.transform import rescale, resize
resizef = lambda x:resize(x, (x.shape[0] // 2, x.shape[1] // 2))
x_train_resized = np.asarray(list(map(resizef,x_train)))
y_train_resized = np.asarray(list(map(resizef,y_train)))
x_test_resized = np.asarray(list(map(resizef,x_test)))

from matplotlib import pyplot as plt
def visualize(img_arr):
    plt.imshow((img_arr*255).reshape(img_arr.shape[:2]).astype(np.uint8),cmap='gray')
    plt.axis('off')
    plt.show()
visualize(y_train_resized[0])

In [ ]:
x = kl.Input(shape=x_train_resized[0].shape)
model = keras.models.Model(x, denseNet_FCN(class_size=1)(x) )
model.summary()
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(model,show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
model.compile(sgd, "binary_crossentropy", ["accuracy"])
model.fit(x_train_resized.astype(np.float32),y_train_resized.astype(np.float32)
          ,batch_size=1,epochs=300,)#callbacks=[hist])